In [ ]:
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
# organize dataset into a useful structure

# create directories
#'/gdrive/MyDrive/5644_ML_Project', 'Dataset_SkinCancer/train'
dataset_home = '/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = '/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		copyfile(src, dst)

KeyboardInterrupt: ignored

In [ ]:
# to get data
train_path = '/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/train/'
print(train_path)
test_path = '/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/test/'
print(test_path)

/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/train/
/gdrive/MyDrive/5644_ML_Project/dataset_dogs_vs_cats/test/


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2) #,horizontal_flip=True)#)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(32,32),
        batch_size=20,
        class_mode='binary',
        subset='training')
validation_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(32,32),
        batch_size=10,
        class_mode='binary',
        subset='validation')
test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(32,32),
        batch_size=10,
        class_mode='binary')

Found 14996 images belonging to 2 classes.
Found 3748 images belonging to 2 classes.
Found 6325 images belonging to 2 classes.


In [ ]:
# CNN model

#def model_1 (Malignant_train_images, Malignant_test_images):
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation = "relu", padding='same', input_shape = (32,32,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation = "relu"),
    #tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation = "relu"),
    #tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation = "relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(80,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])
    # training the model    
#opt = tf.keras.optimizers.Adam(learning_rate=0.01),
opt = keras.optimizers.Adam(learning_rate=0.001)
model.summary()

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

# checkpoint
#filepath="best_weights_p_neuron-{epoch:02d}-{val_acc:.2f}.hdf5"
filepath = r"/gdrive/MyDrive/5644_ML_Project/Models_1/cat_dogs_model_v1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
early_stop = EarlyStopping (monitor='val_acc', mode='max', verbose=0, patience=5)
callbacks_list = [early_stop,checkpoint]

history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 750,epochs = 50, validation_steps = 375, verbose = 1, callbacks=callbacks_list) #

# serialize model to JSON
model_json = model.to_json()
with open("/gdrive/MyDrive/5644_ML_Project/Models_1/cat_dogs_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/gdrive/MyDrive/5644_ML_Project/Models_1/cat_dogs_model_v1.h5")
print("Saved model to disk")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 128)      

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
750/750 [==============================] - 4526s 6s/step - loss: 0.6961 - acc: 0.5435 - val_loss: 0.6584 - val_acc: 0.5859
Epoch 2/50
750/750 [==============================] - 66s 89ms/step - loss: 0.6454 - acc: 0.6299 - val_loss: 0.5978 - val_acc: 0.6945
Epoch 3/50
750/750 [==============================] - 68s 90ms/step - loss: 0.6029 - acc: 0.6747 - val_loss: 0.5776 - val_acc: 0.7012
Epoch 4/50
750/750 [==============================] - 68s 91ms/step - loss: 0.5815 - acc: 0.6940 - val_loss: 0.5440 - val_acc: 0.7276
Epoch 5/50
750/750 [==============================] - 68s 91ms/step - loss: 0.5552 - acc: 0.7145 - val_loss: 0.5316 - val_acc: 0.7327
Epoch 6/50
750/750 [==============================] - 70s 93ms/step - loss: 0.5314 - acc: 0.7302 - val_loss: 0.5149 - val_acc: 0.7375
Epoch 7/50
750/750 [==============================] - 69s 92ms/step - loss: 0.5134 - acc: 0.7510 - val_loss: 0.5000 - val_acc: 0.7577
Epoch 8/50
750/750 [==============================] - 69s 92ms

In [ ]:
loss , accuracy = model.evaluate(test_generator)

235/633 [==========>...................] - ETA: 15s - loss: 0.4121 - acc: 0.8153

UnknownError: ignored